## Houston zip code
Rice Military 77007
Katy 77494
Methods
- Divide houston metropolitan area into 2 counties: Harris county and Fort Bend county

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time
import webdriver
from random import seed, randint, random

In [10]:
def dstr_2_int(dstr):
    return int(dstr[1:].replace(',',''))
def get_p_z_zr_mc(soup):
    soup_p = soup.select('div.ds-chip>div.ds-home-details-chip>div.ds-summary-row-container>div.ds-summary-row-content>div.ds-summary-row>h4>span>span')
    if soup_p:
        if len(soup_p) > 1: # "From 399,000" From is in the first <span>
            Price = dstr_2_int(soup_p[-1].text)
        else:
            Price = dstr_2_int(soup_p[0].text)
    else:
        Price = 0

    soup_zm = soup.select('div#ds-home-values div.ds-expandable-card-section-default-padding>div>div>div>div>span')
    if soup_zm:
        Zestmate = dstr_2_int(soup_zm[0].text)
    else:
        Zestmate = 0
    
    soup_zmr = soup.select('div#ds-rental-home-values span.Text-c11n-8-18-0__aiai24-0.bloUvX')
    if soup_zmr:
        Zestmate_Rent = dstr_2_int(soup_zmr[0].text.replace('/mo', ''))
    else:
        Zestmate_Rent = 0
   
    soup_mc = soup.select('li.ds-data-view-item>div>div.ds-expandable-card-section-default-padding>div.ds-expandable-card>div>h5')
    #('h5.Text-c11n-8-18-0__aiai24-0.StyledHeading-c11n-8-18-0__ktujwe-0.LJxVR.sc-qQmou.bViFbQ')
    if soup_mc:
        Monthly_Cost = dstr_2_int(soup_mc[0].text)
    else:
        Monthly_Cost = 0
    return Price, Zestmate, Zestmate_Rent, Monthly_Cost
def get_school_info(soup):
    soup_sh = soup.select('span#skip-link-schools+div li>div:first-of-type>div>span:first-of-type')
    if soup_sh:
        scores = map(lambda x: x.text, soup_sh)
        scores_sum = 0
        try:
            scores_int = map(lambda x: int(x.text), soup_sh)
            scores_sum = sum(scores_int)
        except:
            pass
        return ','.join(scores), scores_sum
    else:
        return '', 0

def extract_zillow_info(df_out, path_out, n_start=0, driver_type='selenium', sec_sleep=3):
    cols = ['zestmate', 'zestmate_rent', 'monthly_cost', 'school', 'school_sum', 'monthly_cash_flow', 'yearly_return']
    for c in cols:
        df_out[c] = np.nan
    for i, row in df_out[n_start:].iterrows():
        url = row['url']
        # print(url)
        soup = webdriver.get_soup(url, driver_type=driver_type)
        Price, Zestmate, Zestmate_Rent, Monthly_Cost = get_p_z_zr_mc(soup)
        sch_scores, sch_scores_sum = get_school_info(soup)
        # print(Price, Zestmate, Zestmate_Rent, Monthly_Cost)
        # print(sch_scores)
        df_out.loc[i, ['zestmate', 'zestmate_rent', 'monthly_cost', 'school', 'school_sum']] = Zestmate, Zestmate_Rent, Monthly_Cost, sch_scores, sch_scores_sum
        if Zestmate_Rent != 0 and Monthly_Cost != 0:
            df_out.loc[i, 'monthly_cash_flow'] = Zestmate_Rent - Monthly_Cost
            # the yearly return assumes 20% down payment and 2% overall closing fee
            df_out.loc[i, 'yearly_return'] = round((Zestmate_Rent - Monthly_Cost) * 12 / (Price * 0.22), 3)
        if i % 50 == 0:
            df_out.to_csv(path_out)
            print(f'{i} homes have been scaped ....')
        time.sleep(random() * sec_sleep  
        df_out.to_csv(path_out)

### Get all Fort Bend info

In [1]:
# df = pd.read_csv(r'data/dataset_FortBend_2021-01-10.csv')
# cols = [c for c in df.columns if 'photo' not in c]
# df_out = df[cols]
from import_data import load_json_homes
folder = r'/Users/yuchenzhang/Documents/Reps/actor-zillow-api-scraper/apify_storage/datasets/default'
df_out = load_json_homes(folder)

In [6]:

df_out['zestmate'], df_out['zestmate_rent'], df_out['monthly_cost'] = [np.nan, np.nan, np.nan]
df_out['monthly_cash_flow'] = np.nan
df_out['yearly_return'] = np.nan
df_out['school'] = np.nan

In [7]:
# filter out land with no bedroom or bathroom
df_out = df_out[(df_out.bedrooms) > 0 & (df_out.bathrooms > 0)].reset_index()

In [14]:
extract_zillow_info(df_out, './FB_2021_01_16.csv', n_start=78, driver_type='selenium', sec_sleep=3)

100 homes have been scaped ....


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=87.0.4280.141)


In [13]:
df_out[50:]

,index,address,bathrooms,bedrooms,city,community,description,livingArea,neighborhood,price,...,url,yearBuilt,zipcode,zestmate,zestmate_rent,monthly_cost,monthly_cash_flow,yearly_return,school,school_sum
50,64,"6214 Scott Cir, Rosenberg, TX, 77471",2.0,3.0,Rosenberg,None,"This Blakemore II home design features 3 BR, 2...",2019.0,None,270068.0,...,https://www.zillow.com/homedetails/6214-Scott-...,2020.0,77471,0.0,0.0,0.0,NaN,NaN,,0.0
51,65,"2315 Anzio Ct, Missouri City, TX, 77459",3.0,5.0,Missouri City,None,Two story home for sale in Lake Shore Harbour ...,3638.0,None,436900.0,...,https://www.zillow.com/homedetails/2315-Anzio-...,2021.0,77459,0.0,0.0,0.0,NaN,NaN,,0.0
52,66,"6606 High Knoll Dr, Sugar Land, TX, 77479",3.0,4.0,Sugar Land,None,Spacious 4 bedroom two story in Greatwood Knol...,2339.0,None,289900.0,...,https://www.zillow.com/homedetails/6606-High-K...,1992.0,77479,287164.0,2000.0,1636.0,364.0,0.068,"10,8,8",26.0
53,67,"834 Old Oyster Trl, Sugar Land, TX, 77478",4.0,4.0,Sugar Land,None,Nestled on one of the largest premium lots in ...,3033.0,None,620000.0,...,https://www.zillow.com/homedetails/834-Old-Oys...,2008.0,77478,602382.0,4773.0,3462.0,1311.0,0.115,"8,7,7",22.0
54,68,"1210 Brazoswood Pl, Richmond, TX, 77406",6.0,5.0,Richmond,None,Tranquil waterfront home with incredible views...,5276.0,None,799900.0,...,https://www.zillow.com/homedetails/1210-Brazos...,2005.0,77406,761179.0,5194.0,4435.0,759.0,0.052,"9,5,7",21.0
55,69,"3718 Springhill Ln, Sugar Land, TX, 77479",4.0,5.0,Sugar Land,None,Gem of a home located in sought after communit...,3822.0,None,654000.0,...,https://www.zillow.com/homedetails/3718-Spring...,1996.0,77479,647887.0,3830.0,3626.0,204.0,0.017,"9,10,8",27.0
56,70,"56 Schubach Dr, Sugar Land, TX, 77479",4.0,4.0,Sugar Land,None,"LOOKING FOR A TRUE CUSTOM HOME, IN A GATED COM...",NaN,None,690000.0,...,https://www.zillow.com/homedetails/56-Schubach...,1999.0,77479,666945.0,3800.0,3826.0,-26.0,-0.002,"9,10,8",27.0
57,71,"6011 Mound Airy Ct, Sugar Land, TX, 77479",3.0,4.0,Sugar Land,None,Beautiful home located corner lot in New Terri...,3010.0,None,410000.0,...,https://www.zillow.com/homedetails/6011-Mound-...,1990.0,77479,410000.0,2500.0,2314.0,186.0,0.025,"9,10,8",27.0
58,72,"5118 Long Branch Bnd, Fulshear, TX, 77441",5.0,5.0,Fulshear,None,MLS# 76141483 - Built by Highland Homes - Febr...,3752.0,None,525000.0,...,https://www.zillow.com/homedetails/5118-Long-B...,2020.0,77441,511147.0,0.0,2963.0,NaN,NaN,"6,7,7",20.0
59,73,"722 Yaupon Creek Ln, Richmond, TX, 77469",4.0,4.0,Richmond,None,Greatwood Lake is a picturesque single-family ...,2839.0,None,421367.0,...,https://www.zillow.com/homedetails/722-Yaupon-...,2021.0,77469,0.0,0.0,2378.0,NaN,NaN,"5,8,8",21.0


### Get Katy House Info

In [27]:
df_2021 = pd.read_excel(r'./target_asset/2021.xlsx')
df_2021_Katy = df_2021[df_2021.Area == 'Katy']
extract_zillow_info(df_2021_Katy[:2], './2021_Katy.csv', n_start=0, driver_type='selenium', sec_sleep=2)